In [28]:
# import libraries
import pandas as pd
import polars as pl
import io
import requests as reqs
import json
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

In [18]:
token = "TOKEN"
host = "https://firms.modaps.eosdis.nasa.gov/api/country/csv/"

source = "VIIRS_SNPP_NRT/"
country = "IDN/"
today = "2023-08-19"
day_range = "1"

url = (host + token + source + country + day_range + "/" + today)

get_data = reqs.get(url)

if get_data.status_code == 200:
    csv_content = io.StringIO(get_data.text)
    viirs_df = pl.read_csv(csv_content)
    print(viirs_df)
else:
    print(f"Failed to fetch data, status code: {get_data.status_code}")

shape: (1_611, 15)
┌────────────┬──────────┬───────────┬────────────┬───┬─────────┬────────────┬──────┬──────────┐
│ country_id ┆ latitude ┆ longitude ┆ bright_ti4 ┆ … ┆ version ┆ bright_ti5 ┆ frp  ┆ daynight │
│ ---        ┆ ---      ┆ ---       ┆ ---        ┆   ┆ ---     ┆ ---        ┆ ---  ┆ ---      │
│ str        ┆ f64      ┆ f64       ┆ f64        ┆   ┆ str     ┆ f64        ┆ f64  ┆ str      │
╞════════════╪══════════╪═══════════╪════════════╪═══╪═════════╪════════════╪══════╪══════════╡
│ IDN        ┆ -8.9224  ┆ 140.80379 ┆ 332.79     ┆ … ┆ 2.0NRT  ┆ 304.14     ┆ 5.0  ┆ D        │
│ IDN        ┆ -8.85717 ┆ 140.89622 ┆ 342.07     ┆ … ┆ 2.0NRT  ┆ 306.92     ┆ 4.79 ┆ D        │
│ IDN        ┆ -8.85545 ┆ 140.95421 ┆ 335.37     ┆ … ┆ 2.0NRT  ┆ 305.06     ┆ 3.4  ┆ D        │
│ IDN        ┆ -8.85492 ┆ 140.95776 ┆ 338.72     ┆ … ┆ 2.0NRT  ┆ 304.19     ┆ 3.3  ┆ D        │
│ …          ┆ …        ┆ …         ┆ …          ┆ … ┆ …       ┆ …          ┆ …    ┆ …        │
│ IDN        ┆ -7.469

In [21]:
def fetch_viirs_data(today: str, day_range: str) -> pl.DataFrame:
    """
    Retrieves VIIRS active fires data from the NASA endpoint for a given date and range.

    Parameters:
    - today (str): The specific date for which the data is to be retrieved, in the format "YYYY-MM-DD".
    - day_range (str): The range of days for which the data is to be retrieved.

    Returns:
    - pl.DataFrame: A Polars DataFrame containing the VIIRS data if successful, or None if an error occurred.

    Examples:
    >>> get_viirs_data("2023-08-19", "1")
    """

    try:
        token = "TOKEN"
        host = "https://firms.modaps.eosdis.nasa.gov/api/country/csv/"
        source = "VIIRS_NOAA20_NRT/"
        country = "IDN/"

        url = (host + token + source + country + day_range + "/" + today)

        get_data = requests.get(url)
        get_data.raise_for_status() # Raises an HTTPError if the HTTP request returned an unsuccessful status code

        csv_content = io.StringIO(get_data.text)
        viirs_df = pl.read_csv(csv_content)
        
        return viirs_df

    except requests.RequestException as e:
        print(f"An error occurred while making the HTTP request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [22]:
today = "2023-08-19"
day_range = "1"

viirs_df = fetch_viirs_data(today, day_range)

viirs_df

country_id,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
str,f64,f64,f64,f64,f64,str,i64,i64,str,str,str,f64,f64,str
"""IDN""",-8.84913,140.95351,346.36,0.74,0.76,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",295.9,12.29,"""D"""
"""IDN""",-8.75335,140.75838,351.47,0.76,0.77,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",297.47,13.69,"""D"""
"""IDN""",-8.75253,140.76527,342.92,0.76,0.77,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",296.15,13.69,"""D"""
"""IDN""",-8.68832,140.72372,348.13,0.77,0.77,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",296.53,13.7,"""D"""
"""IDN""",-8.68563,140.72203,347.56,0.77,0.77,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",296.35,13.1,"""D"""
"""IDN""",-8.67539,140.65164,326.04,0.77,0.78,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",295.92,4.07,"""D"""
"""IDN""",-8.54813,140.95401,326.58,0.74,0.76,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",293.48,6.67,"""D"""
"""IDN""",-8.24709,140.54141,326.25,0.78,0.78,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",292.17,10.21,"""D"""
"""IDN""",-8.24624,140.54842,334.1,0.78,0.78,"""2023-08-19""",313,1,"""VIIRS""","""n""","""2.0NRT""",292.62,10.59,"""D"""
